In [ ]:
!pip install glove-python-binary
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 948 kB 4.0 MB/s 


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [39]:
from glove import Glove,Corpus
from gensim.models import Word2Vec

class GV:
    def __init__(self,vector_size, window_size, lr = 0.05, workers = 4,data=[]):
        self.vector_size = vector_size
        self.window_size = window_size
        self.learning_rate = lr
        self.workers = workers

        self.data = data
        self.model = None
        self.vocab = None
        self.corpus=Corpus()
        if data!=[]:
            self.corpus.fit(self.data, window=self.window_size)
  
    def load_glove(self,path):
        self.model=Glove.load(path)
        self.vocab=list(self.model.dictionary.keys())

    def train_glove(self,num_epochs,model_path=""):
        # self.corpus.fit(self.data, window=self.window_size)
        glove = Glove(no_components=self.vector_size, learning_rate=self.learning_rate) 
        start = time.perf_counter()
        glove.fit(self.corpus.matrix, epochs=num_epochs, no_threads=self.workers, verbose=True)
        end = time.perf_counter()
        glove.add_dictionary(self.corpus.dictionary)
        self.model=glove
        self.vocab=list(self.model.dictionary.keys())
        if model_path != "":
            self.model.save(model_path)
        return end-start
    def get_top_similar_words(self, word, num = 5):
        most_similar = np.array(self.model.most_similar(word, number = num+1)).transpose()[0]
        return most_similar


    def check_synonyms_in_model(self,target, words, num):
        count = 0
        similar_words = self.get_top_similar_words(target, num)
        for word in words:
            if word in similar_words:
                count += 1
        return count



class W2V:
    def __init__(self, vector_size, window_size, sg = 1, lr = 0.05, workers = 4,data=[]):
        self.vector_size = vector_size
        self.window_size = window_size
        self.sg = sg
        self.learning_rate = lr
        self.workers = workers

        self.data = data
        self.model = None
        self.vocab = None

    def train_word2vec(self, num_epochs, model_path = ""):
        print("\n\nTraining word2vec model")
        # Create Skip Gram model ( sg=1 )  or CBOW ( sg=0 )
        start = time.perf_counter()

        word2vec = Word2Vec(sentences=self.data,size=self.vector_size,
                            window=self.window_size,sg=self.sg,min_count =0,
                            # epochs=num_epochs,
                            workers=self.workers,alpha=self.learning_rate)
                            
            # self.data, self.vector_size,self.window_size,
            #                 self.sg,num_epochs,self.workers, self.learning_rate)

        end = time.perf_counter()

        print("Finished training word2vec model\n")

        self.model = word2vec
        self.vocab = self.model.wv.index2word
        # self.vocab = self.model.wv.index_to_key
        if model_path != "":
            self.model.save(model_path)

        return end - start

    def check_one_word(self, word):
        a = []
        for i in self.vocab:
            a.append(self.model.wv.similarity(word, i))
        return a

  
    def delete_unknown_words(self, words):

        known_words = [word for word in words if word in self.vocab]
        unknown_words = list(set(words) - set(known_words))
        return known_words, unknown_words

    def get_top_similar_words(self, word, num = 5):
        most_similar = np.array(self.model.wv.most_similar(word, topn = num)).transpose()[0]
        return most_similar


    def check_synonyms_in_model(self,target, words, num):
        count = 0
        similar_words = self.get_top_similar_words(target, num)
        for word in words:
            if word in similar_words:
                count += 1
        return count

    def load_model(self, path):
        self.model = Word2Vec.load(path)
        self.vocab = self.model.wv.index2word
        # self.vocab = self.model.wv.index_to_key

In [40]:
import numpy as np
import pandas as pd
import time
# from nlp_models.py import *
from nltk.tokenize import sent_tokenize, word_tokenize

pd.options.mode.chained_assignment = None

# from patent_dataset import df

# full_text = df.full_text


def delete_words_not_in_vocab(data, col, model1):
    index = [i for i, target in enumerate(data[col]) if target in model1.vocab]
    filtered_data = data.iloc[index]
    return filtered_data


# path = r'C:\Users\Chen\Desktop\Masters_degree\NLP'  # PC Chen
path = r'C:/Users/Chen/Documents/Masters_degree/word2vec_vs_glove'  # Laptop Chen
# full_text = pd.read_pickle(r"C:\Users\Chen\Desktop\Masters_degree\NLP\full_text.pkl")

# model = W2V(300, 2, 1, 0.05, 4)
# model.prepare_data(full_text, limit_data=100)
# train_time = model.train_word2vec(num_epochs=30, model_path=path + r'/output/word2vec_model')

# print(f"training the model took : {train_time:0.4f} sec \n")

glove_model = GV(300, 2, 0.05, 4)
glove_model.load_glove(path='/content/drive/MyDrive/Colab Notebooks/output/glove_model')

w2v_model = W2V(300, 2, 1, 0.05, 4)
w2v_model.load_model('/content/drive/MyDrive/Colab Notebooks/output/word2vec_model')



In [45]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/synonyms.csv').dropna()
df = delete_words_not_in_vocab(df, "lemma", w2v_model)
df = df.drop("part_of_speech", axis=1)
df.synonyms = [i.replace("|", ";").split(";") for i in df.synonyms]

df = df.dropna().reset_index().drop("index", axis=1)
data = df.groupby("lemma").sum().reset_index().dropna()
data = delete_words_not_in_vocab(data, "lemma", w2v_model)


In [65]:
print("\n\n")
count_w2v = []
count_glove = []
for i in range(len(data)):
    count_w2v.append(w2v_model.check_synonyms_in_model(data.loc[i, "lemma"], data.loc[i, "synonyms"], 5))
    count_glove.append(glove_model.check_synonyms_in_model(data.loc[i, "lemma"], data.loc[i, "synonyms"], 5))
    print("\r", end='')
    print(f"{i}/7283 number of matches in: w2v:{sum(count_w2v)}",f" glove:{sum(count_glove)}",end='', flush=True)

    # print()




7282/7283 number of matches in: w2v:409  glove:273